In [11]:
import uuid
import asyncio
import nest_asyncio
# Apply nest_asyncio to allow running asyncio in Jupyter
nest_asyncio.apply()

thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        "thread_id": thread_id,
    }
}

from neurapolis_cognitive_architecture.agent import graph
from langchain_core.messages import HumanMessage

async def process_user_input(user_input):
    
    inputs = {"messages": [HumanMessage(content=user_input)]}
    async for chunk in graph.astream(inputs, config=config, stream_mode="updates"):
        node_name = list(chunk.keys())[0]
        print(f"---------- Update from node {node_name} ---------")
        print(chunk[node_name])
async def main():
    while True:
        user_input = input("\nEnter your question (or 'exit' to quit): ")
        if user_input.lower() == "exit":
            break
        await process_user_input(user_input)

if __name__ == "__main__":
    asyncio.run(main())


---------- Update from node agent ---------
{'messages': [AIMessage(content='Guten Tag! Ich bin der KI-Assistent für Neurapolis, das Ratsdokumentenmanagementsystem der Stadt Freiburg. Wie kann ich Ihnen heute helfen? Haben Sie Fragen zu städtischen Angelegenheiten, Verordnungen oder aktuellen Projekten in Freiburg?', additional_kwargs={'usage': {'prompt_tokens': 1041, 'completion_tokens': 85, 'total_tokens': 1126}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, response_metadata={'usage': {'prompt_tokens': 1041, 'completion_tokens': 85, 'total_tokens': 1126}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run-236a915d-bce5-49c6-98d9-deba1212dbdf-0', usage_metadata={'input_tokens': 1041, 'output_tokens': 85, 'total_tokens': 1126})]}
